In [ ]:
from tensor import Tensor
from loss import MSELoss
import nn
import numpy as np
from helper import unbroadcast
import activations as act
from visualization import draw_dot
import time

n = 1000
# Create (100, 3) input dataset with values from a uniform distribution
X_data = np.random.uniform(low=-10, high=10, size=(n, 3)).astype(np.float32)
Y_data = (X_data[:, 0] + X_data[:, 1]*X_data[:, 2]).reshape(-1, 1)

# Normalize data
X_mean, X_std = X_data.mean(axis=0), X_data.std(axis=0)
Y_mean, Y_std = Y_data.mean(),    Y_data.std()

X_data = (X_data - X_mean) / X_std
Y_data = (Y_data - Y_mean) / Y_std

# Wrap with your Tensor class
x_train = Tensor(X_data[:int(0.8*n)], requires_grad=False)
y_train = Tensor(Y_data[:int(0.8*n)], requires_grad=False)

x_test = Tensor(X_data[int(0.8*n):], requires_grad=False)
y_test = Tensor(Y_data[int(0.8*n):], requires_grad=False)

print(x_train.shape, y_train.shape)

In [ ]:
model = nn.Sequential(
    nn.Linear(3, 64, requires_grad=True),  # first linear
    nn.ReLU(),                                         # activation
    nn.Linear(64, 16, requires_grad=True),
    nn.ReLU(),
    nn.Linear(16 ,4, requires_grad=True),
    nn.ReLU(),
    nn.Linear(4, 1, requires_grad=True)
)

In [ ]:
lr = 1e-2
epochs = 10000

# calculating the time taken for training
start_time = time.time()

for epoch in range(1, epochs+1):
    model.zero_grad()

    # forward
    y_pred = model(x_train)
    loss   = MSELoss(y_pred, y_train)
    loss.backward()

    # update
    for p in model.parameters():
        p.data -= lr * p.grad

    if epoch % 200 == 0:
        test_pred = model(x_test)
        test_loss = MSELoss(test_pred, y_test)
        print(f"Epoch {epoch:4d} | train={loss.data:.6f} | test={test_loss.data:.6f}")

# final
end_time = time.time()
final_pred = model(x_test)
print("Final test loss:", MSELoss(y_test, final_pred).data)
print("Time taken for training: " + str(end_time - start_time) + " seconds")

In [ ]:
a = Tensor([1.0, 3.0, 6.0], requires_grad=False)
a.data = (a.data - X_mean) / X_std
a = a.reshape(1, -1)  # reshape to match input shape
out = model(a)
out.data = out.data * Y_std + Y_mean  # denormalize output
print(out)

In [2]:
from tensor import Tensor
import numpy as np
test_A = Tensor([[1.0, 3.0, 6.0], [2, 4, 6]], requires_grad=True)
print(f"Test tensor: {test_A}")
print("###########################")
##
print("Test 1:")
sum_1 = test_A.sum(axis=None, keepdims=False)
print(sum_1)    # should be equal to Tensor(data = 22.0), shape()
sum_1.backward()
print(f"test_A.grad={test_A.grad}")     # grad should be [[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
print(f"sum_1.grad={sum_1.grad}")       # grad should be 1.0 as sum_1.backward() initializes a scalar's grad to 1.0
print("Passed successfully!")
print("###########################")
##
print("Test 2:")
test_A.zero_grad()
sum_2 = test_A.sum(axis=None, keepdims=True)
print(sum_2)    # should be equal to Tensor(data = 22.0, shape=(1, 1))
sum_2.backward(grad=np.ones_like(sum_2.data))
print(f"test_A.grad={test_A.grad}")     # grad should be [[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
print(f"sum_2.grad={sum_2.grad}")       # grad should be [[1.0]]
print("Passed successfully!")
print("###########################")
##
print("Test 3:")
test_A.zero_grad()
sum_3 = test_A.sum(axis=0, keepdims=False)
print(sum_3)
sum_3.backward(grad=np.array([1.0, 1.0, 1.0]))
print(f"test_A.grad={test_A.grad}")     # grad should be [[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
print(f"sum_3.grad={sum_3.grad}")
print("Passed successfully!")
print("###########################")
##
print("Test 4:")
test_A.zero_grad()
sum_4 = test_A.sum(axis=0, keepdims=True)
print(sum_4)
sum_4.backward(grad=np.ones_like(sum_4.data))
print(f"test_A.grad={test_A.grad}")     # grad should be [[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
print(f"sum_4.grad={sum_4.grad}")
print("Passed successfully!")
print("###########################")
##
print("test 5:")
test_A.zero_grad()
sum_5 = test_A.sum(axis=1, keepdims=False)
print(sum_5)
sum_5.backward(grad=np.array([1.0, 1.0]))
print(f"test_A.grad={test_A.grad}")     # grad should be [[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
print(f"sum_5.grad={sum_5.grad}")
print("Passed successfully!")
print("###########################")
##
print("Test 6:")
test_A.zero_grad()
sum_6 = test_A.sum(axis=1, keepdims=True)
print(sum_6)
sum_6.backward(grad=np.ones_like(sum_6.data))
print(f"test_A.grad={test_A.grad}")     # grad should be [[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
print(f"sum_6.grad={sum_6.grad}")
print("Passed successfully!")
print("###########################")

Test tensor: Tensor(data=[[1. 3. 6.]
 [2. 4. 6.]], shape=(2, 3), requires_grad=True, label='')
###########################
Test 1:
Tensor(data=22.0, shape=(), requires_grad=True, label='')
test_A.grad=[[1. 1. 1.]
 [1. 1. 1.]]
sum_1.grad=1.0
Passed successfully!
###########################
Test 2:
Tensor(data=[[22.]], shape=(1, 1), requires_grad=True, label='')
test_A.grad=[[1. 1. 1.]
 [1. 1. 1.]]
sum_2.grad=[[1.]]
Passed successfully!
###########################
Test 3:
Tensor(data=[ 3.  7. 12.], shape=(3,), requires_grad=True, label='')
test_A.grad=[[1. 1. 1.]
 [1. 1. 1.]]
sum_3.grad=[1. 1. 1.]
Passed successfully!
###########################
Test 4:
Tensor(data=[[ 3.  7. 12.]], shape=(1, 3), requires_grad=True, label='')
test_A.grad=[[1. 1. 1.]
 [1. 1. 1.]]
sum_4.grad=[[1. 1. 1.]]
Passed successfully!
###########################
test 5:
Tensor(data=[10. 12.], shape=(2,), requires_grad=True, label='')
test_A.grad=[[1. 1. 1.]
 [1. 1. 1.]]
sum_5.grad=[1. 1.]
Passed successfully!
#######

In [2]:
from tensor import Tensor
import numpy as np

# Create test tensor for mean operations
test_A = Tensor([[1.0, 3.0, 6.0],
                 [2.0, 4.0, 6.0]], requires_grad=True)
print("Test tensor for mean:", test_A)
print("###########################")

# Mean Test 1: scalar mean, no keepdims
print("Mean Test 1:")
mean_1 = test_A.mean()
print(mean_1)      # Expect Tensor(data=3.666..., shape=(), ...)
mean_1.backward()
print(f"test_A.grad={test_A.grad}")   # Expect each entry = 1/6 (~0.1667)
print(f"mean_1.grad={mean_1.grad}")   # Expect 1.0
print("Passed successfully!\n###########################\n")

# Mean Test 2: scalar mean with keepdims=True
test_A.zero_grad()
print("Mean Test 2:")
mean_2 = test_A.mean(axis=None, keepdims=True)
print(mean_2)      # Expect shape=(1,1), data=3.666...
mean_2.backward(grad=np.ones_like(mean_2.data))
print(f"test_A.grad={test_A.grad}")   # Expect each entry = 1/6
print(f"mean_2.grad={mean_2.grad}")   # Expect [[1.0]]
print("Passed successfully!\n###########################\n")

# Mean Test 3: mean over axis=0, no keepdims
test_A.zero_grad()
print("Mean Test 3:")
mean_3 = test_A.mean(axis=0, keepdims=False)
print(mean_3)      # Expect Tensor(data=[1.5,3.5,6.], shape=(3,), ...)
mean_3.backward(grad=np.array([1.0,1.0,1.0]))
print(f"test_A.grad={test_A.grad}")   # Expect each column grad=1/2
print(f"mean_3.grad={mean_3.grad}")   # Expect [1.0,1.0,1.0]
print("Passed successfully!\n###########################\n")

# Mean Test 4: mean over axis=0, keepdims=True
test_A.zero_grad()
print("Mean Test 4:")
mean_4 = test_A.mean(axis=0, keepdims=True)
print(mean_4)      # Expect shape=(1,3), data=[[1.5,3.5,6.]]
mean_4.backward(grad=np.ones_like(mean_4.data))
print(f"test_A.grad={test_A.grad}")   # Expect each column grad=1/2
print(f"mean_4.grad={mean_4.grad}")   # Expect [[1.0,1.0,1.0]]
print("Passed successfully!\n###########################\n")

# Mean Test 5: mean over axis=1, no keepdims
test_A.zero_grad()
print("Mean Test 5:")
mean_5 = test_A.mean(axis=1, keepdims=False)
print(mean_5)      # Expect Tensor(data=[(1+3+6)/3, (2+4+6)/3] = [3.333...,4.], ...)
mean_5.backward(grad=np.array([1.0,1.0]))
print(f"test_A.grad={test_A.grad}")   # Expect each row grad=1/3
print(f"mean_5.grad={mean_5.grad}")   # Expect [1.0,1.0]
print("Passed successfully!\n###########################\n")

# Mean Test 6: mean over axis=1, keepdims=True
test_A.zero_grad()
print("Mean Test 6:")
mean_6 = test_A.mean(axis=1, keepdims=True)
print(mean_6)      # Expect shape=(2,1), data=[[3.333...],[4.]]
mean_6.backward(grad=np.ones_like(mean_6.data))
print(f"test_A.grad={test_A.grad}")   # Expect each row grad=1/3
print(f"mean_6.grad={mean_6.grad}")   # Expect [[1.0],[1.0]]
print("Passed successfully!\n###########################\n")

# Create test tensor for reshape operations
test_B = Tensor([[1.0, 2.0, 3.0],
                 [4.0, 5.0, 6.0]], requires_grad=True)
print("Test tensor for reshape:", test_B)
print("###########################")

# Reshape Test 1: (2,3) -> (3,2)
print("Reshape Test 1:")
r1 = test_B.reshape(3, 2)
print(r1)         # Expect Tensor(data=[[1,2],[3,4],[5,6]], shape=(3,2), ...)
r1.backward(grad=np.ones_like(r1.data))
print(f"test_B.grad={test_B.grad}")  # Expect all ones
print("Passed successfully!\n###########################\n")

# Reshape Test 2: flatten to (6,)
print("Reshape Test 2:")
test_B.zero_grad()                              # reset gradients
r2 = test_B.reshape(6)                          # forward
print(r2)                                       # Tensor(data=[1 2 3 4 5 6], shape=(6,))
r2.backward(grad=np.ones_like(r2.data))         # backward with ones
print(f"test_B.grad={test_B.grad}")             # Expect [[1 1 1], [1 1 1]]
print("Passed successfully!")


Test tensor for mean: Tensor(data=[[1. 3. 6.]
 [2. 4. 6.]], shape=(2, 3), requires_grad=True, label='')
###########################
Mean Test 1:
Tensor(data=3.6666667461395264, shape=(), requires_grad=True, label='')
test_A.grad=[[0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667]]
mean_1.grad=1.0
Passed successfully!
###########################

Mean Test 2:
Tensor(data=[[3.6666667]], shape=(1, 1), requires_grad=True, label='')
test_A.grad=[[0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667]]
mean_2.grad=[[1.]]
Passed successfully!
###########################

Mean Test 3:
Tensor(data=[1.5 3.5 6. ], shape=(3,), requires_grad=True, label='')
test_A.grad=[[0.5 0.5 0.5]
 [0.5 0.5 0.5]]
mean_3.grad=[1. 1. 1.]
Passed successfully!
###########################

Mean Test 4:
Tensor(data=[[1.5 3.5 6. ]], shape=(1, 3), requires_grad=True, label='')
test_A.grad=[[0.5 0.5 0.5]
 [0.5 0.5 0.5]]
mean_4.grad=[[1. 1. 1.]]
Passed successfully!
#####################